#**Kiểm thử Vietnamese Speech Recognition with Wav2Vec**
Nhận diện giọng nói tiếng Việt với Wav2Vec

Đồ án chuyên ngành **Khoa học dữ liệu**

---

**Họ và tên SV 1:** Nguyễn Thành Phát

**MSSV SV 1:** 2000006273

---

**Họ và tên SV 2:** Trần Thị Bé Quyên

**MSSV SV 2:** 2000005977

---
#**(CTRL + F9 để chạy tất cả)**

In [ ]:
!pip install transformers datasets gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 9.6 MB/s eta 0:0

In [ ]:
from transformers import AutoModelForCTC
from transformers import AutoConfig
from transformers import AutoFeatureExtractor
from transformers import Wav2Vec2Processor
from transformers import AutoTokenizer
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_checkpoint = "phatjk/wav2vec2-large-vi"

config = AutoConfig.from_pretrained(model_checkpoint)

tokenizer_type = config.model_type if config.tokenizer_class is None else None
config = config if config.tokenizer_class is not None else None


feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)


tokenizer = AutoTokenizer.from_pretrained(
  model_checkpoint,
  config=config,
  tokenizer_type=tokenizer_type,
  unk_token="[UNK]",
  pad_token="[PAD]",
  word_delimiter_token="|",
)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
model = AutoModelForCTC.from_pretrained(
    model_checkpoint,
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
).to(device)


`use_fast` is set to `True` but the tokenizer class does not have a fast version.  Falling back to the slow version.


In [ ]:
from datasets import load_dataset, load_metric, Audio
data = load_dataset("mozilla-foundation/common_voice_11_0", "vi", split="validation[:100]")

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 2525it [00:00, 70519.96it/s]


Generating validation split: 0 examples [00:00, ? examples/s]

...


Reading metadata...: 1237it [00:00, 84658.06it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 11476it [00:00, 121584.16it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 337it [00:00, 55754.20it/s]


In [ ]:
data = data.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
def VietnameseSpeechRecognition(arrTensor):
  inputs = processor(arrTensor, sampling_rate=16000, return_tensors="pt")
  inputs = inputs.to(device)
  with torch.no_grad():
      logits = model(**inputs).logits
      # Ket qua du doan
      predicted_ids = torch.argmax(logits, dim=-1)
      transcription = processor.batch_decode(predicted_ids)
      return transcription[0]

In [ ]:
# Chon audio de test
import IPython.display as ipd
import numpy as np
import random
audio_sample = data[19]["audio"]["array"]
ipd.Audio(data=audio_sample, autoplay=False, rate=16000)

In [ ]:
print(VietnameseSpeechRecognition(audio_sample))

quân mờ mắt ra mới biết mình vừa nằm mơ


In [ ]:
import os

import numpy as np

import gradio as gr
import scipy.signal as sps

def ReSampleAudio(data,current_rate,new_rate=16000):
  samples = round(len(data) * float(new_rate) / current_rate)
  return sps.resample(data, samples)

def reverse_audio(audio):
    sr, dataArr = audio
    dataArr = ReSampleAudio(dataArr,sr)
    inputs = torch.tensor(dataArr)
    return VietnameseSpeechRecognition(inputs)


demo = gr.Interface(fn=reverse_audio,
                    inputs="microphone",
                    outputs="text",
                    examples=[
                    data[18]["path"],
                    data[92]["path"],
                    data[55]["path"],
                    data[47]["path"],
                    data[26]["path"],
                    ],
                     cache_examples=True,
                    )


demo.launch()


Caching examples at: '/content/gradio_cached_examples/13'
Caching example 1/5
Caching example 2/5
Caching example 3/5
Caching example 4/5
Caching example 5/5
Caching complete

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>